### Transformata Hougha w przestrzeni ab - Domowe
Przestrzeń ρ,θ nie jest jedyną przestrzenią w której punkt odpowiada parametrom prostej. Np. można spróbować wykorzystać tradycyjne równanie prostej:

y=ax+b

W tej przestrzeni reprezentacją pęku prostych jest prosta. 

Zadanie: napisać funkcję, która jako argument przyjmuje obraz (binarny) oraz parametry:

- aMin - minimalna wartość parametru a
- aMax - maksymalna  wartość parametru a
- aSkok - skok parametru a
- bMin - minimalna wartość parametru b
- bMax  - maksymalna  wartość parametru b
- bSkok - skok parametru b
Jako wynik ma zwrócić macierz przestrzeni Hougha ab.

Uwagi:

- zadanie może wyglądać na skomplikowane ale tak na prawdę wymaga tylko starannego przemyślenia
- najważniejszy jest problem "adresowania" macierzy H. Można do tego wykorzystać dwa wektory A i B ze wszystkimi możliwymi wartościami jakie parametry a i b mogę przyjąć (w podanych zakresach z podanym skokiem). Wówczas indeksy tych wektorów będą współrzędnymi w macierzy H. Do stworzenia wektorów wykorzystaj funkcję _np.arange_
- na podstawie rozmiaru wektorów A i B (własność _shape_) należy stworzyć wyzerowaną macierz H (funkcja _np.zeros_)
- dla każdego punktu krawędzi na obrazie należy:
    - używająć wektora A dla każdej z jego wartości a obliczyć odpowiednią wartość b - indeksy kolejnych wartości a (a_idx) będą jedną z współrzędnych macierzy H, natomiast drugą współrędną mozna uzyskać jako b_idx = np.argmin(abs(B - b)).
        - każde a_idx i b_idx wyznaczą punkt w macierzy H, który należy zinkrementować
- działanie funkcji należy przetestować na punktach generowanych w 'edytorku' z punktu "Transformacja Hougha dla małej liczby punktów". Proponowane parametry funkcji:
    - aMin: -5
    - aMax:  5
    - aSkok: 0.05 
    - bMin: -100
    - bMax:  100
    - bSkok: 1

Zastanów się w przypadku jakich prostych reprezentacja ab nie sprawdzi się.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.transform import hough_line, hough_line_peaks
import os
import requests

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/11_Hough/'

fileNames = ["dom.png", "kwadraty.png", "lab112.png"]
for fileName in fileNames:
    path = 'img/' + fileName
    if not os.path.exists(path):
        r = requests.get(url + fileName, allow_redirects=True)
        open(path, 'wb').write(r.content)

In [ ]:
def show(imgs, titles=None):
    if len(imgs) == 1:
        plt.figure(figsize=(10, 10))
        plt.imshow(imgs[0], cmap='gray')
        if titles is not None:
            plt.title(titles[0])
        plt.axis('off')
        plt.show()
        return

    _, axis = plt.subplots(1, len(imgs), figsize=(7*len(imgs), 7))
    for i, ax in enumerate(axis):
        ax.imshow(imgs[i], cmap='gray')
        if titles is not None:
            ax.set_title(titles[i])
        ax.axis('off')

    plt.show()

In [ ]:
def hough_transform(img, a_params, b_params):
    # Stworzenie wektorów a i b z podanymi parametrami
    A = np.arange(*a_params)
    B = np.arange(*b_params)

    # Stworzenie macierzy H wypełnionej zerami
    H = np.zeros((len(A), len(B)))

    # Pętla po punktach na obrazie
    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            if img[y, x] > 0:
                # Dla każdej wartości a oblicz odpowiednią wartość b
                for a_idx, a_val in enumerate(A):
                    b_val = y - a_val * x
                    # Znajdź indeks b_idx odpowiadający b_val
                    b_idx = np.argmin(np.abs(B - b_val))
                    # Zinkrementuj wartość w macierzy H
                    H[a_idx, b_idx] += 1
    
    H_normalized = (H - np.min(H)) / (np.max(H) - np.min(H)) * 255
    return H_normalized.astype(np.uint8)

In [ ]:
# Przykładowe użycie funkcji
img = np.zeros((64, 64), dtype=np.uint8)

for i in range(0, 16, 3):
    img[i*4, 16+i] = 1

img[40, 50] = 1

aMin, aMax, aSkok = -5, 5, 0.05
bMin, bMax, bSkok = -100, 100, 1
a_params = [aMin, aMax, aSkok]
b_params = [bMin, bMax, bSkok]

result_matrix = hough_transform(img, a_params, b_params)
show([img, result_matrix], ['Obraz binarny', 'Wynik transformacji Hougha'])

In [ ]:
img = cv2.imread('img/kwadraty.png', cv2.IMREAD_GRAYSCALE)
img = cv2.medianBlur(img, 3)
img = cv2.Canny(img, 50, 200)
show([img], ['Obraz binarny'])

aMin, aMax, aSkok = -np.pi / 2, np.pi / 2, 0.01
bMin, bMax, bSkok = -100, 100, 1
a_params = [aMin, aMax, aSkok]
b_params = [bMin, bMax, bSkok]

result_matrix = hough_transform(img, a_params, b_params)
show([img, result_matrix], ['Obraz binarny', 'Wynik transformacji Hougha'])